In [179]:
import tools as tl
from selenium.webdriver.support.select import Select
import requests
import pandas as pd
import configparser
import matplotlib.pyplot as plt

In [ ]:
def get_config_data(**kwargs):
    """
    This function reads a configuration file and returns the value of a specified field.
    
    :param path: The path to the configuration file that needs to be read
    :return: the data from the specified field and value in the configuration file located at the
    specified path.
    """
    #Abrindo o arquivo de configurações
    arq = configparser.RawConfigParser()
    arq.read(CONFIG_PATH)

    #Pegando os dados do arquivo
    data = arq.get(kwargs['field'], kwargs['value'])

    return data


In [180]:
# Função para substituir os emails pelo nome do cliente
def replace_emails_with_names(email, customer_dict):
    
    for name, emails in customer_dict.items():
        if (email.lower() in [em.lower() for em in emails]):
            return name
    return email

In [181]:
def create_customer_dict(customers_list, df_all):
    customer_dict = {}

    for customer in customers_list: 
        df_filt = df_all.loc[df_all['To'].str.contains(customer)]
        customer_dict[customer] = df_filt['To'].value_counts().index.to_list()
        #df_filt.to_excel(f'{OUTPUT_PATH}\{customer}.xlsx', index=False)
    
    return customer_dict

In [ ]:
def replace_custom_names(customer_names):
    diff_custom = {
        'aguia':'aguiasecuritizadora',
        'bombinhassummer':'bombinhassummerbeach', 
        'dutoclean': 'dutocleanbauru',
        'fabricenter': 'fabricenter470',
        'fazcomunicacao': 'fazcomunicacaovisual',
        'galli': 'galliatacadista',
        'impacto': 'impactoesports',
        'jcleletronica': 'jcleletronicaindustrial',
        'mailinspector': 'filter.mailinspector',
        'plantfort': 'plantfortfertil',
        'primedigital': 'primemidiadigital',
        'quinta': 'quintadonino',
        'raysolar': 'raysolarbrasil',
        'reset': 'resetmairinque',
        'revar': 'revarcondicionado',
        'rodoflex': 'rodoflexpneus',
        'sany': 'sanydobrasil',
        'supernagai': 'supermercadosnagai',
        'agrotecagro': 'agrotecagroquimica',
        'barulhinhobomchips': 'barulhinhobom',
        'marcospaulo': 'marcospauloimoveis',
        'cortedobra': 'cortedobraseguranca',
        'ortobaby': 'ortobabycalcados',
        'arinteli': 'arinteligenciatributaria',
        
    }   

    for diff_name in diff_custom.keys():
        list_index = customer_names.index(diff_name)
        customer_names[list_index] = diff_custom[diff_name]
    
    return customer_names

In [1]:
diff_custom = {
    'aguia':'aguiasecuritizadora',
    'bombinhassummer':'bombinhassummerbeach', 
    'dutoclean': 'dutocleanbauru',
    'fabricenter': 'fabricenter470',
    'fazcomunicacao': 'fazcomunicacaovisual',
    'galli': 'galliatacadista',
    'impacto': 'impactoesports',
    'jcleletronica': 'jcleletronicaindustrial',
    'mailinspector': 'filter.mailinspector',
    'plantfort': 'plantfortfertil',
    'primedigital': 'primemidiadigital',
    'quinta': 'quintadonino',
    'raysolar': 'raysolarbrasil',
    'reset': 'resetmairinque',
    'revar': 'revarcondicionado',
    'rodoflex': 'rodoflexpneus',
    'sany': 'sanydobrasil',
    'supernagai': 'supermercadosnagai',
    'agrotecagro': 'agrotecagroquimica',
    'barulhinhobomchips': 'barulhinhobom',
    'marcospaulo': 'marcospauloimoveis',
    'cortedobra': 'cortedobraseguranca',
    'ortobaby': 'ortobabycalcados',
    'arinteli': 'arinteligenciatributaria',
    
}


In [ ]:
def get_config_data(path, **kwargs):
    
    #Abrindo o arquivo de configurações
    arq = configparser.RawConfigParser()
    arq.read(path)
    
    #Pegando os dados do arquivo
    data = arq.get(kwargs['field'], kwargs['value'])
    
    return data

In [182]:
def replace_numeric_chars(string):
    replacements = {'0': 'o', '1': 'i', '3': 'e', '4': 'a', '5': 's', '7': 't', '8': 'b'}
    for char, replacement in replacements.items():
        if string not in ['5stintas','c3']:
            string = string.replace(char, replacement)
    return string

In [183]:
URL = 'https://filter.mailinspector.com.br/login/index.php'
API_URL = 'https://filter.mailinspector.com.br/login/mailLogViewer.php'
OUTPUT_PATH = r'C:\Users\Vitor Augusto\Documents\Programas\abreus_network_bot_report\files'
user ='admin'
senha = 'N6wq@Ks72'

In [184]:
tl.clean_dir(OUTPUT_PATH)

Limpando arquivos gerais
customers_emails.xlsx removido com sucesso.
log_view.csv removido com sucesso.
log_view.txt removido com sucesso.
original.csv removido com sucesso.


### 1 Etapa

- Acessar o portal
- Salvar a lista de clientes 
- Salvar os cookies para passar na API

In [185]:
driver = tl.create_driver(headless=False)
driver.get(URL)

In [186]:
tl.clicking(element='clicando em login', path='email', by='name', driver=driver).click()
tl.clicking(element='passando o user', path='email', by='name', driver=driver).send_keys(user)

tl.clicking(element='clicando em senha', path='password', by='name', driver=driver).click()
tl.clicking(element='passando a senha', path='password', by='name', driver=driver).send_keys(senha)

tl.clicking(element='clicando no botao login', path="//input[@type='submit']", driver=driver).click()

>>>Sucesso clicando em login>>>
>>>Sucesso passando o user>>>
>>>Sucesso clicando em senha>>>
>>>Sucesso passando a senha>>>
>>>Sucesso clicando no botao login>>>


In [187]:
tl.clicking(element='Selecionando Linked Accounts', path="//*[text()='Linked Accounts']", driver=driver).click() 

>>>Sucesso Selecionando Linked Accounts>>>


In [188]:
drop_down = tl.clicking(element='selecionando o selector com todos os clientes', path='hijackName', by='name', driver=driver)
drop_down_obj = Select(drop_down)

>>>Sucesso selecionando o selector com todos os clientes>>>


In [190]:
customers_list = [opt.text for opt in drop_down_obj.options]
customers_list.pop(0)
customers_list = [em.lower() for em in customers_list]
customers_list = list(map(lambda char: replace_numeric_chars(char), customers_list))
customers_list = replace_custom_names(customers_list)

In [191]:
cookies = tl.get_cookies(driver)

In [192]:
driver.quit()

### 2 Etapa
- Configurar os paramentros da API
- Acessar os reports dos clientes via API
- salvar como arquivo csv para facilitar a modelagem

In [193]:
headersList = {
 "Cookie": f"PHPSESSID={cookies['PHPSESSID']}; lang=pt_BR",
 "Referer": "https://filter.mailinspector.com.br/login/mailLogViewer.php", 
}

response = requests.get('https://filter.mailinspector.com.br/login/mailLogViewer.php?from=05%2F17%2F2023&to=05%2F18%2F2023&page=-1', headers=headersList)
result = response.text

In [194]:
with open(f'{OUTPUT_PATH}\log_view.txt', 'w') as f:
    f.write(result)

In [195]:
df = pd.read_csv(f'{OUTPUT_PATH}\log_view.txt')
df.to_csv(f'{OUTPUT_PATH}\original.csv', index=False, sep=',')
df.head()

,Date,From,To,Subject,Action,Score,Source IP
0,2023-05-17 23:59:40,reminders@facebookmail.com,marcelo2@superpanelao.com.br,=?UTF-8?B?QXR1YWxpemHDp8O1ZXMgcGE=?=? =?UTF-8?...,Sent,0.0,216.55.103.238
1,2023-05-17 23:59:40,reminders@facebookmail.com,marcelo2@superpanelao.com.br,=?UTF-8?B?QXR1YWxpemHDp8O1ZXMgcGE=?=? =?UTF-8?...,Sent,NaN,66.220.144.144
2,2023-05-17 23:59:29,bounces+1686153-0001-crm4=breitkopf.com.br@ema...,crm4@breitkopf.com.br,NaN,Rejected,NaN,167.89.93.240
3,2023-05-17 23:59:27,bounces+8038336-a8a4-elieser.fernandes=csimont...,elieser.fernandes@csimontagens.com.br,NaN,Rejected,NaN,149.72.227.0
4,2023-05-17 23:59:27,bounces+1686153-ec68-crm3=breitkopf.com.br@ema...,crm3@breitkopf.com.br,NaN,Rejected,NaN,167.89.93.240


### 4 Etapa
- Organizar o arquivo extraido da API
- Alterar a coluna 'To' que são os emails, para os nomes dos clientes

In [196]:
customer_dict = {}
for customer in customers_list:
    customer_dict[customer] = [email for email in df['To'] if (email.endswith('@'+customer.lower()+'.com.br')) or (email.endswith('@'+customer.lower()+'.com'))]

In [197]:
#customer_dict = create_customer_dict(customers_list, df)
df_customer = pd.DataFrame({"Cliente":list(customer_dict.keys()),
                            "Emails":list(customer_dict.values())})

df_customer.to_excel(f'{OUTPUT_PATH}\customers_emails.xlsx', index=False)

In [198]:
df_all = pd.DataFrame({'Date': df['Date'], 
                       'From':df['From'], 
                       'To': df['To'].apply(replace_emails_with_names, customer_dict=customer_dict), 
                       'Action': df['Action']})
df_all.head()

,Date,From,To,Action
0,2023-05-17 23:59:40,reminders@facebookmail.com,superpanelao,Sent
1,2023-05-17 23:59:40,reminders@facebookmail.com,superpanelao,Sent
2,2023-05-17 23:59:29,bounces+1686153-0001-crm4=breitkopf.com.br@ema...,breitkopf,Rejected
3,2023-05-17 23:59:27,bounces+8038336-a8a4-elieser.fernandes=csimont...,csimontagens,Rejected
4,2023-05-17 23:59:27,bounces+1686153-ec68-crm3=breitkopf.com.br@ema...,breitkopf,Rejected


In [199]:
df_all.to_csv(f'{OUTPUT_PATH}\log_view.csv', sep=',', index=False)

In [200]:
df_errados = df_all.loc[df_all['To'].str.contains('@')]
df_errados['To'].value_counts()

To
marilia@mariliavalenca.com.br         185
advogada@viviane.adv.br                98
raquelmarques@unixsaude.com.br         84
cccerqueira@unixsaude.com.br           71
vendas@greaselubrificantes.com.br      69
                                     ... 
dfk10ervo08p@outlookweb.com.br          1
hostmaster@outlookweb.com.br            1
trv4zuq9hd99z@outlookweb.com.br         1
uqe77r3nm66u7gkv@outlookweb.com.br      1
faturadigital@itau.com.br               1
Name: count, Length: 1045, dtype: int64

In [201]:
error_list = []

for cust in df_errados['To'].drop_duplicates():
    split_str = cust.split('@')[1]
    if "@"+split_str not in error_list:
        error_list.append("@"+split_str)

In [202]:
error_list

['@mariliavalenca.com.br',
 '@unixsaude.com.br',
 '@lardasfitas.com.br',
 '@pentafixacoes.com.br',
 '@nfendd.com.br',
 '@basesc.eng.br',
 '@viviane.adv.br',
 '@4takes.com.br',
 '@greaselubrificantes.com.br',
 '@gressusseguros.com.br',
 '@outlook.de',
 '@greaseespecial.com.br',
 '@dimasdona.com.br',
 '@elder-th.eti.br',
 '@freto.com',
 '@facebookmail.com',
 '@otto.ind.br',
 '@deschampscontainer.com.br',
 '@pirosolucoes.com.br',
 '@VIVIANE.ADV.BR',
 '@jallesmachado.com',
 '@VITORIAATACAREJO.COM.BR',
 '@casadecarnesaraujo.com.br',
 '@valius.com.br',
 '@SUPERPANELAO.COM.BR',
 '@gruponazarao.com.br',
 '@grupopetropolis.com.br',
 '@cideestagio.com.br',
 '@abreubusiness.com.br',
 '@duffles.net.br',
 '@adv.oabsp.org.br',
 '@zancapel.com.br',
 '@sompo.com.br',
 '@delegatelists.com',
 '@bauducco.com.br',
 '@simpleslocadora.com.br',
 '@gmail.com',
 '@renovaagroibitinga.com.br',
 '@delegatlist.com',
 '@sa-east-1.amazonses.com',
 '@patiocorina.com.br',
 '@reseteletronicaindustrial.com.br',
 '@renat

### 5 Etapa
- Extrair do arquivo organizado os dados a serem mostrado na dashboard

In [203]:
#contas que mais madaram email x ação realizada
df_filt = df_all.loc[df_all['To']=='marcelo']
serie = df_filt['From'].value_counts()
email_series = serie.loc[serie>1].index.to_list()
df_from = df_filt.loc[df_filt['From'].isin(email_series)]
qtd_series = df_from['From'].value_counts()
df_tres = pd.DataFrame({'From': qtd_series.index, 'Qtd': qtd_series.values, 'Actions': df_from.drop_duplicates(subset=['From'])['Action']})
df_tres

,From,Qtd,Actions
